In [1]:
%matplotlib notebook

import matplotlib.pyplot as plt
import numpy as np
from numpy import sin, cos
import numba as nb
from scipy.optimize import fsolve
from numpy.linalg import inv

In [14]:
# power flow function (equality constraints)
def gfun(x, u, p):
  
    VM3 = x[0]
    VA3 = x[1]
    VA2 = x[2]
    P1  = x[3]
    
    VM1 = u[0]
    P2 = u[1]
    VM2 = u[2]
    
    VA1 = p[0]
    P3 = p[1]
    Q3 = p[2]

    # intermediate quantities
    VA23 = VA2 - VA3
    VA31 = VA3 - VA1
    VA32 = VA3 - VA2
    VA13 = VA1 - VA3
    
    F1 = 4.0*VM2*VM2 + VM2*VM3*(-4*cos(VA23) + 10*sin(VA23)) - P2
    F2 = (8.0*VM3*VM3 + VM3*VM1*(-4*cos(VA31) + 5*sin(VA31))
          + VM3*VM2*(-4*cos(VA32) + 10*sin(VA32)) + P3)
    F3 = (15.0*VM3*VM3 + VM3*VM1*(-4*sin(VA31) - 5*cos(VA31))
          + VM3*VM2*(-4*sin(VA32) - 10*cos(VA32)) + Q3)
    F4 = 4.0*VM1*VM1 + VM1*VM3*(-4*cos(VA13) + 5*sin(VA13)) - P1
    
    
    return np.array([F1, F2, F3, F4])

# cost function
def cfun(x, u, p):

    VM3 = x[0]
    VA3 = x[1]
    P1 = x[3]

    VM1 = u[0]
    P2 = u[1]

    VA1 = p[0]

    VA13 = VA1 - VA3
    
    w1 = 1.0
    w2 = 1.0
    
    cost = P1 + P2
    
    return cost

In [17]:
# Jacobians and gradients

def gfun_x(x, u, p):

    
    VM3 = x[0]
    VA3 = x[1]
    VA2 = x[2]
    
    VM1 = u[0]
    P2 = u[1]
    VM2 = u[2]
    
    VA1 = p[0]
    P3 = p[1]
    Q3 = p[2]

    # intermediate quantities
    VA23 = VA2 - VA3
    VA31 = VA3 - VA1
    VA32 = VA3 - VA2
    
    J = np.zeros((4, 4))
    
    #F1
    J[0, 0] =  VM2*(10*sin(VA2 - VA3) - 4*cos(VA2 - VA3))
    J[0, 1] =  VM2*VM3*(-4*sin(VA2 - VA3) - 10*cos(VA2 - VA3))
    J[0, 2] =  VM2*VM3*(4*sin(VA2 - VA3) + 10*cos(VA2 - VA3))
    J[0, 3] =  0
    #F2
    J[1, 0] =  VM1*(-5*sin(VA1 - VA3) - 4*cos(VA1 - VA3)) + VM2*(-10*sin(VA2 - VA3) - 4*cos(VA2 - VA3)) + 16.0*VM3
    J[1, 1] =  VM1*VM3*(-4*sin(VA1 - VA3) + 5*cos(VA1 - VA3)) + VM2*VM3*(-4*sin(VA2 - VA3) + 10*cos(VA2 - VA3))
    J[1, 2] =  VM2*VM3*(4*sin(VA2 - VA3) - 10*cos(VA2 - VA3))
    J[1, 3] =  0
    #F3
    J[2, 0] =  VM1*(4*sin(VA1 - VA3) - 5*cos(VA1 - VA3)) + VM2*(4*sin(VA2 - VA3) - 10*cos(VA2 - VA3)) + 30.0*VM3
    J[2, 1] =  VM1*VM3*(-5*sin(VA1 - VA3) - 4*cos(VA1 - VA3)) + VM2*VM3*(-10*sin(VA2 - VA3) - 4*cos(VA2 - VA3))
    J[2, 2] =  VM2*VM3*(10*sin(VA2 - VA3) + 4*cos(VA2 - VA3))
    J[2, 3] =  0
    #F4
    J[3, 0] =  VM1*(5*sin(VA1 - VA3) - 4*cos(VA1 - VA3))
    J[3, 1] =  VM1*VM3*(-4*sin(VA1 - VA3) - 5*cos(VA1 - VA3))
    J[3, 2] =  0
    J[3, 3] =  -1


    return J

def gfun_u(x, u, p):
    
    VM3 = x[0]
    VA3 = x[1]
    VA2 = x[2]
    
    VM1 = u[0]
    P2 = u[1]
    VM2 = u[2]
    
    VA1 = p[0]
    P3 = p[1]
    Q3 = p[2]

    # intermediate quantities
    VA23 = VA2 - VA3
    VA31 = VA3 - VA1
    VA32 = VA3 - VA2
    
    J = np.zeros((4, 3))
    
    #F1
    J[0, 0] =  0
    J[0, 1] =  -1
    J[0, 2] =  8.0*VM2 + VM3*(10*sin(VA2 - VA3) - 4*cos(VA2 - VA3))
    #F2
    J[1, 0] =  VM3*(-5*sin(VA1 - VA3) - 4*cos(VA1 - VA3))
    J[1, 1] =  0
    J[1, 2] =  VM3*(-10*sin(VA2 - VA3) - 4*cos(VA2 - VA3))
    #F3
    J[2, 0] =  VM3*(4*sin(VA1 - VA3) - 5*cos(VA1 - VA3))
    J[2, 1] =  0
    J[2, 2] =  VM3*(4*sin(VA2 - VA3) - 10*cos(VA2 - VA3))
    #F4
    J[3, 0] =  8.0*VM1 + VM3*(5*sin(VA1 - VA3) - 4*cos(VA1 - VA3))
    J[3, 1] =  0
    J[3, 2] =  0

    return J


def cfun_x(x, u, p):
    
    VM3 = x[0]
    VA3 = x[1]

    VM1 = u[0]
    P2 = u[1]

    VA1 = p[0]

    VA13 = VA1 - VA3
    
    w1 = 1.0
    w2 = 1.0
    
    grad = np.zeros(4)
    grad[0] =  0
    grad[1] =  0
    grad[2] =  0
    grad[3] =  w1

    
    return grad

def cfun_u(x, u, p):
    
    VM3 = x[0]
    VA3 = x[1]

    VM1 = u[0]
    P2 = u[1]

    VA1 = p[0]

    VA13 = VA1 - VA3
    
    w1 = 1.0
    w2 = 1.0
    
    grad = np.zeros(3)
    grad[0] =  0
    grad[1] =  w2
    grad[2] =  0
    
    return grad

Initialize script with same initial conditions as in the paper

In [18]:
# initial parameters
x = np.zeros(4)
u = np.zeros(3)
p = np.zeros(3)

# this is given by the problem data, but might be "controlled" via OPF
u[0] = 1.0 #VM1
u[1] = 1.7 #P2
u[2] = 1.0 #VM2

# these parameters are fixed through the computation
p[0] = 0.0 #VA1, slack angle
p[1] = 2.0 #P3
p[2] = 1.0 #Q3


# initial guess
x[0] = 1.0 #VM3
x[1] = 0.0 #VA3
x[2] = 0.0 #VA2
x[3] = 0.5


# print initial guesses
print(x)
print(u)

[1.  0.  0.  0.5]
[1.  1.7 1. ]


In [19]:
# POWER FLOW ALGO

def powerflow(x, u, p):
    
    sol = fsolve(gfun, x, args=(u,p,))
    return sol

print(powerflow(x, u, p))

print(np.linalg.cond(gfun_x(x, u, p)))


[ 0.88186783 -0.00094814  0.1349708   0.47671095]
44.553094929145


In [25]:
# Reduced gradient iteration

max_iter = 100
xk = np.copy(x)
uk = np.copy(u)

for i in range(max_iter):
    
    # power flow
    xk = powerflow(xk, uk, p)

    # lambda calculation
    J_x = gfun_x(xk, uk, p)
    G_x = cfun_x(xk, uk, p)
    print("condition jacobian:", np.linalg.cond(J_x))
    lam = -np.dot(inv(np.transpose(J_x)), G_x)
    
    # gradient cost function
    J_u = gfun_u(xk, uk, p)
    G_u = cfun_u(xk, uk, p)
    
    grad_c = G_u + np.dot(np.transpose(J_u), lam)
    print("Norm of gradient: ", np.linalg.norm(grad_c))
    
    # evaluate cost function
    print("Cost function: ", cfun(xk, uk, p))    
    
    # compute step
    alpha = 0.12
    uk = uk - alpha*grad_c

condition jacobian: 39.59958561633352
Norm of gradient:  0.7038831242740287
Cost function:  2.1767109512499543
condition jacobian: 46.68308341866992
Norm of gradient:  0.5531510420686726
Cost function:  2.1598296133612314
condition jacobian: 44.71393962254119
Norm of gradient:  0.4450119712664789
Cost function:  2.149087109860759
condition jacobian: 49.48699914092949
Norm of gradient:  0.3506100402295171
Cost function:  2.1398376900170497
condition jacobian: 48.99354430244175
Norm of gradient:  0.2865642394328765
Cost function:  2.1329741612923963
condition jacobian: 52.204220536784455
Norm of gradient:  0.23911203526530678
Cost function:  2.127220777411127
condition jacobian: 52.614602175128354
Norm of gradient:  0.2081087508231566
Cost function:  2.122449802647705
condition jacobian: 54.89049527130433
Norm of gradient:  0.186821075858742
Cost function:  2.1183014665515123
condition jacobian: 55.76733748285113
Norm of gradient:  0.17233726046432105
Cost function:  2.1146440807183913
c